In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

from pprint import pprint

import database
import helper

# Data Search

In [ ]:
client = database.connect()
db = client.JiraRepos

print(db.list_collection_names())
collection = db['Apache']

In [ ]:
data = collection.find_one()
summary = data['fields']['summary']

print(len(summary))
print(summary)

In [ ]:
data['fields']['issuetype']['name']

In [ ]:
# db.users.aggregate(
#    [ { $sample: { size: 3 } } ]
# )

die Issue types haben übergreift andere namen
Bevor man also nahc den Issue types sucht muss man eine IN Funktion von MongoDB verwenden und dieser dann alle möglichen werte für z.B. Story eine Joras geben
Diese Informationen finden sich in jira_issuetype_thematic_analysis.json

In [ ]:
results = collection.find_one({
    "$and":[
        {"fields.issuetype.name": "Story"},
        {"$expr": {"$gt": [{"$strLenCP": "$fields.summary"}, 70] }} 
        ]
    })

In [ ]:
pprint(results)

# Ticket Creation

In [ ]:
df = pd.read_csv('./data/output.csv')

In [ ]:
## Get the Summary entries for a specific issue type
# new_df = helper.addIssueTypeToField(df, 'Summary')
# new_UserStory_df = new_df[new_df['issue_type'] == 'User Story']
# new_UserStory_df.head(10)

In [ ]:
# 284325 from Jira could be a good example for the Description completness Prompt!!
sample_id = 284325
jira = "Jira"
evolutionStep = 1

In [ ]:
sample = df[(df["issue_id"] == sample_id) & (df["jira"] == jira)]

ticket = helper.createTicket(sample, evolutionStep)
ticket

In [ ]:
helper.saveTicket('summary', ticket, evolutionStep, jira, sample_id)